In [9]:
import tdmclient.notebook
from pylsl import StreamInlet, resolve_stream
import numpy as np
import time

In [10]:
def connectStream():
        # first resolve an EEG stream on the lab network
        print("looking for an EEG stream...")
        streams = resolve_stream('type', 'EEG')


        # create a new inlet to read from the stream
        inlet = StreamInlet(streams[0])

        print('connection found')
        return inlet

In [11]:
await tdmclient.notebook.start()

In [12]:
try : inlet = connectStream()
except : print('No stream found')

looking for an EEG stream...
connection found


Code pour suivre la ligne 

In [18]:
%%run_python

#onevent startup
maxSteer = 20
# speed_left = 400
# speed_right= 400
speed = 0
steerL = 0
tmp=0
   

@onevent
def sendSpeed(s):
    global speed
    speed = s

@onevent
def prox():
    global speed, motor_left_target, motor_right_target, maxSteer, steerL, tmp
    right = prox_ground_delta[1]
    left = prox_ground_delta[0]
    tmp = maxSteer * left - maxSteer * right

    steerL = tmp // 50

    motor_left_target = speed + steerL
    motor_right_target = speed - steerL


In [23]:
send_event('sendSpeed',0)

In [16]:
while True:
    # get a new sample (you can also omit the timestamp part if you're not
    # interested in it)
    sample, timestamp = inlet.pull_sample()
    if timestamp:
        alpha = sample[0]
        # alpha = valeur actuelle - baseline
        # donc si alpha < 0 : plus concentré = on augmente la vitesse
        # sinon on réduit la vitesse
        if alpha > 0:
            speed = max(0,speed - alpha)
        else:
            speed = min(speed - alpha,400)
        
        print(f'Alpha : {alpha}')
        print(f'Speed : {speed}\n')
        send_event("sendSpeed",int(speed))
        

Alpha : 0.024904897436499596
Speed : 0

Alpha : -2.2758896350860596
Speed : 2.2758896350860596

Alpha : -3.9613356590270996
Speed : 6.237225294113159

Alpha : -3.603689193725586
Speed : 9.840914487838745

Alpha : 1.5266227722167969
Speed : 8.314291715621948

Alpha : -5.037415504455566
Speed : 13.351707220077515

Alpha : -4.533919811248779
Speed : 17.885627031326294

Alpha : -4.083740234375
Speed : 21.969367265701294

Alpha : -2.1970784664154053
Speed : 24.1664457321167

Alpha : 24.729219436645508
Speed : 0

Alpha : 46.524200439453125
Speed : 0

Alpha : 50.873355865478516
Speed : 0

Alpha : 53.70646667480469
Speed : 0

Alpha : 54.57367706298828
Speed : 0

Alpha : 43.62922286987305
Speed : 0

Alpha : 35.92595672607422
Speed : 0

Alpha : 35.68257141113281
Speed : 0

Alpha : 37.8681755065918
Speed : 0

Alpha : 33.852779388427734
Speed : 0

Alpha : 30.7632999420166
Speed : 0

Alpha : 36.3780517578125
Speed : 0

Alpha : 37.300079345703125
Speed : 0

Alpha : 30.810514450073242
Speed : 0

Alph

KeyboardInterrupt: 

Error in callback <function _post_run_cell at 0x000002617CA3D760> (for post_run_cell), with arguments args (<ExecutionResult object at 2617f141400, execution_count=16 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 2617f1423f0, raw_cell="while True:
    # get a new sample (you can also o.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/33770/Documents/GitHub/brain_kart/Robot/Hackatech.ipynb#X10sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: object of type 'float' has no len()

In [46]:
%%run_aseba

#var target_speed
var left
var right
# mean and vari(ance) calculation vars
var mini
var maxi
var mean
var vari
# actual value and n(ormalized)dev(iation)
var p1
var p2
var ndev1
var ndev2
# pi controller vars
var ireg
var preg
var pcoeff
var icoeff

var SIGMA
var speed_left
var speed_right

#onevent startup
  mini=1023
  maxi=0
  mean=512
  vari=512
  ireg=0

  SIGMA = 100

  speed_left = 400
  speed_right = 400


onevent prox

   p1=prox.ground.delta[0]
   p2=prox.ground.delta[1]
   callsub statistics 
   call math.muldiv(ndev1,SIGMA,p1-mean,vari)
   call math.muldiv(ndev2,SIGMA,p2-mean,vari)

   if abs(ndev1) < SIGMA then
      preg=60*ndev1/100    
      ireg=ireg+10*preg/30
      motor.left.target=speed_left+(preg+ireg)            
      motor.right.target=speed_right-(preg+ireg)     

   elseif abs(ndev2)<SIGMA then
      preg=60*ndev2/100    
      ireg=ireg+10*preg/30
      motor.left.target=speed_left+(preg+ireg)            
      motor.right.target=speed_right-(preg+ireg)  

   else    
      ireg=0                
      motor.left.target=1*ndev2/2
      motor.right.target=-1*ndev2/2
   
   end

   


sub statistics
   call math.max(maxi,maxi,p1)
   call math.min(mini,mini,p1)        
   if maxi-mini>400 then
      call math.muldiv(vari,45,maxi-mini,100)
      mean=(mini+maxi)/2
   end
